In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

### a) Create a new Spark Session with new SparkConfig

In [3]:
sc.stop()
spark.stop()

In [4]:
from pyspark import SparkConf, SparkContext
# setMaster sets spark ContextManager which is loca[cpu cores]
config = SparkConf().setMaster('local[4]').setAppName("PySparkSession")
sc = SparkContext(conf=config)

In [5]:
sc

<SparkContext master=local[4] appName=PySparkSession>

### b) Create new instance of Spark SQL session and define new DataFrame using sales_data_sample.csv dataset.

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLSession").getOrCreate()

In [7]:
spark

In [8]:
sales_df = spark.read.csv('file:///home/hadoop/Downloads/sales_data_sample.csv', header=True, inferSchema=True)
sales_df.show(5)

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped| 

In [9]:
sales_df.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

### c) Find the shape of DataFrame.

In [10]:
rows = sales_df.count()
columns = len(sales_df.columns)
print(f'Shape of dataframe is: ({rows}, {columns})')

Shape of dataframe is: (2823, 25)


### d) Find the Summary of DataFrame for all numerical data columns.

In [42]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

numerical_columns = [field.name for field in sales_df.schema.fields if not isinstance(field.dataType, StringType)]
sales_df.select(numerical_columns).summary().show()

# INSIGHT: summary of the DataFrame for numerical columns shows count, min vlaue, max value, and quartile values

+-------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|       ORDERNUMBER|  QUANTITYORDERED|         PRICEEACH|  ORDERLINENUMBER|             SALES|            QTR_ID|          MONTH_ID|           YEAR_ID|              MSRP|
+-------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|              2823|             2823|              2823|             2823|              2823|              2823|              2823|              2823|              2823|
|   mean|10258.725115125753|35.09280906836698| 83.65854410201929|6.466170740347148|  3553.88907190932|2.7176762309599716|7.0924548352816155|2003.8150903294368|100.71555083244775|
| stddev|  92.0854775957196| 9.74144273706958|20.174276527840536| 4.22584096469094|1841.8651057401842| 1.

### e) Identify and handle missing or null values in the columns.

In [12]:
sales_df.select([count(when(isnull(col), col)).alias(col) for col in sales_df.columns]).show()
# ADDRESSLINE2, STATE, POSTALCODE - all have NULL values in them - this might need to be handled [dont drop]

+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|ADDRESSLINE2|CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|          0|              0|        0|              0|    0|        0|     0|     0|       0|      0|          0|   0|          0|           0|    0|           0|        2521|   0| 1486|        76|      0|        0|       

In [13]:
# handling null values with empty string
sales_df1 = sales_df.fillna('')

### f) Calculate the total revenue generated per country by combining the columns QUANTITYORDERED and PRICEEACH using Spark DataFrame operations?

In [43]:
queryResult = sales_df1.select(
                col('COUNTRY'), (col('QUANTITYORDERED')*col('PRICEEACH')).alias('REVENUESUM')
              ).groupBy(['COUNTRY']).agg(round(sum('REVENUESUM'), 3).alias('TOTALREVENUE'))
queryResult.show()

+-----------+------------+
|    COUNTRY|TOTALREVENUE|
+-----------+------------+
|     Sweden|    174264.1|
|Philippines|    80291.17|
|  Singapore|    227985.5|
|    Germany|   178689.08|
|     France|   919257.85|
|    Belgium|    94528.88|
|    Finland|    268714.7|
|      Italy|   309402.87|
|     Norway|    246115.8|
|      Spain|  1021705.97|
|    Denmark|   192747.63|
|    Ireland|    43237.24|
|        USA|  2986425.21|
|         UK|   413203.34|
|Switzerland|    93344.91|
|     Canada|   193504.34|
|      Japan|   153076.69|
|  Australia|   521598.46|
|    Austria|   172793.05|
+-----------+------------+



### g) Determine the top 5 products with the highest total sales revenue using Spark DataFrame?

In [15]:
# from pyspark.sql.functions import *
# from pyspark.sql.types import *

In [16]:
queryResult = sales_df1.select(
                col('PRODUCTLINE'), (col('SALES')).alias('TOTALREVENUE')
              ).groupBy(['PRODUCTLINE']).agg(round(sum('TOTALREVENUE'), 3).alias('TOTALREVENUE'))\
              .orderBy(['TOTALREVENUE'], ascending=False)
queryResult.limit(5).show()

+----------------+------------+
|     PRODUCTLINE|TOTALREVENUE|
+----------------+------------+
|    Classic Cars|  3919615.66|
|    Vintage Cars|  1903150.84|
|     Motorcycles|  1166388.34|
|Trucks and Buses|  1127789.84|
|          Planes|   975003.57|
+----------------+------------+



### h) Find the average order quantity for each product using groupBy and agg operations?

In [17]:
queryResult = sales_df1.select(
                col('PRODUCTLINE'), col('QUANTITYORDERED')
              ).groupBy(['PRODUCTLINE']).\
              agg(round(avg('QUANTITYORDERED'), 3).alias('AVG_QUANTITYORDERED'))

queryResult.show()

+----------------+-------------------+
|     PRODUCTLINE|AVG_QUANTITYORDERED|
+----------------+-------------------+
|     Motorcycles|             35.236|
|    Vintage Cars|              34.71|
|           Ships|             34.731|
|Trucks and Buses|             35.804|
|    Classic Cars|             35.152|
|          Trains|             35.221|
|          Planes|             35.056|
+----------------+-------------------+



### i) Using Spark DataFrame, filter orders where the SALES value exceeds $10,000 and sort the results by the ORDERDATE column?

In [45]:
#sales_df1.filter(col('SALES')>10000).show()
from pyspark.sql.functions import to_timestamp
sales_df1.withColumn('ORDERDATE', to_timestamp(col('ORDERDATE'), 'MM/dd/yyyy H:mm'))\
            .select(col('ORDERNUMBER'), col('SALES'), col('ORDERDATE'))\
            .filter(col('SALES')>10000)\
            .orderBy('ORDERDATE').show()

+-----------+-------+-------------------+
|ORDERNUMBER|  SALES|          ORDERDATE|
+-----------+-------+-------------------+
|      10127|11279.2|2003-06-03 00:00:00|
|      10150|10993.5|2003-09-19 00:00:00|
|      10247|10606.2|2004-05-05 00:00:00|
|      10304|10172.7|2004-10-11 00:00:00|
|      10312|11623.7|2004-10-21 00:00:00|
|      10322|12536.5|2004-11-04 00:00:00|
|      10333|11336.7|2004-11-18 00:00:00|
|      10339|10758.0|2004-11-23 00:00:00|
|      10375|10039.6|2005-02-03 00:00:00|
|      10388|10066.6|2005-03-03 00:00:00|
|      10403|11886.6|2005-04-08 00:00:00|
|      10405|11739.7|2005-04-14 00:00:00|
|      10406|10468.9|2005-04-15 00:00:00|
|      10407|14082.8|2005-04-22 00:00:00|
|      10412|11887.8|2005-05-03 00:00:00|
|      10424|12001.0|2005-05-31 00:00:00|
+-----------+-------+-------------------+



### j) Filter out rows where the STATUS is 'Cancelled' and calculate the total sales from the remaining orders?

In [19]:
sales_df1.filter(col('STATUS') != 'Cancelled').agg(round(sum('SALES'), 3).alias('TOTAL SALES')).show()

+-----------+
|TOTAL SALES|
+-----------+
| 9838141.37|
+-----------+



### k) Use Spark Data Frame transformations to derive the yearly sales for each customer (CUSTOMERNAME) based on the ORDERDATE column?

In [20]:
queryResult = sales_df1.withColumn('ORDERDATE', to_timestamp(col('ORDERDATE'), 'MM/dd/yyyy H:mm'))\
              .select(col('CUSTOMERNAME'), year('ORDERDATE').alias('YEAR'), col('SALES'))\
              .groupBy(['CUSTOMERNAME', 'YEAR'])\
              .agg(round(sum('SALES'), 3).alias('AVG_QUANTITYORDERED'))\
              .orderBy(['CUSTOMERNAME', 'YEAR'])

queryResult.show()

+--------------------+----+-------------------+
|        CUSTOMERNAME|YEAR|AVG_QUANTITYORDERED|
+--------------------+----+-------------------+
|      AV Stores, Co.|2003|           51017.92|
|      AV Stores, Co.|2004|          106789.89|
|        Alpha Cognac|2003|           55349.32|
|        Alpha Cognac|2005|           15139.12|
|  Amica Models & Co.|2004|           94117.26|
|Anna's Decoration...|2003|           88983.71|
|Anna's Decoration...|2005|           65012.42|
|   Atelier graphique|2003|            16560.3|
|   Atelier graphique|2004|            7619.66|
|Australian Collec...|2003|           37878.55|
|Australian Collec...|2004|           12334.82|
|Australian Collec...|2005|           14378.09|
|Australian Collec...|2003|           60135.84|
|Australian Collec...|2004|          140859.57|
|Australian Gift N...|2003|           37739.09|
|Australian Gift N...|2005|           21730.03|
|  Auto Assoc. & Cie.|2004|           64834.32|
|    Auto Canal Petit|2004|           79

### l) Add a new column to the DataFrame that categorizes orders as 'High', 'Medium', or 'Low' sales based on the SALES value?

In [21]:
from pyspark.sql.window import Window
from pyspark.sql.functions import ntile

windowSpec = Window.orderBy('SALES')
sales_df1.withColumn('quantile',ntile(3).over(windowSpec))\
         .select(col('ORDERNUMBER'), col('SALES'), col('quantile'))\
         .withColumn(
             'SALES_CATEGORY',
             when(col('quantile')==1, 'Low')\
             .when(col('quantile')==2, 'Medium')\
             .when(col('quantile')==3, 'High')
         )\
         .select(col('ORDERNUMBER'), col('SALES'), col('SALES_CATEGORY'))\
         .show()

+-----------+------+--------------+
|ORDERNUMBER| SALES|SALES_CATEGORY|
+-----------+------+--------------+
|      10425|482.13|           Low|
|      10407|541.14|           Low|
|      10408|553.95|           Low|
|      10280| 577.6|           Low|
|      10419|640.05|           Low|
|      10264| 651.8|           Low|
|      10420|652.35|           Low|
|      10214| 683.8|           Low|
|      10304| 694.6|           Low|
|      10344| 703.6|           Low|
|      10110| 710.2|           Low|
|      10135| 717.4|           Low|
|      10114|721.44|           Low|
|      10358| 728.4|           Low|
|      10375|733.11|           Low|
|      10193|759.46|           Low|
|      10203| 777.0|           Low|
|      10409|785.64|           Low|
|      10281| 813.2|           Low|
|      10156| 820.4|           Low|
+-----------+------+--------------+
only showing top 20 rows



### m) Assume , If you have another DataFrame with customer demographic data, how would you perform a join to compute the total sales per demographic group?

In [22]:
customer_demographic_data = spark.read.csv('file:///home/hadoop/Downloads/sales_data_customer_demographic.csv', header=True, inferSchema=True)
joined_df = sales_df1.join(customer_demographic_data, sales_df1.CUSTOMERNAME==customer_demographic_data.CUSTOMERNAME, "inner")
joined_df.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+--------------------+---------+-------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|        CUSTOMERNAME|ESTD_YEAR|NUM_EMPLOYEES|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------

### n) Can you implement a cumulative distribution function (CDF) over the SALES value for each CUSTOMERNAME? What insights can you gather from analyzing the CDF distribution for each customer?

In [46]:
#from pyspark.sql.functions

#sales_df1.groupBy(['CUSTOMERNAME'])\
#         .agg(
#            round(sum('SALES'), 3).alias('SUM'),
#            round(avg('SALES'), 3).alias('MEAN'),
#            round(expr(f"percentile_approx({'SALES'}, 0.5)"), 3).alias('MEDIAN'),
#            round(stddev('SALES'), 3).alias('STDDEV'),
#            round(count('SALES'), 3).alias('COUNT'),
#            round(max('SALES'), 3).alias('MAX'),
#            round(min('SALEs'), 3).alias('MIN')
#         )\
#         .show()

from pyspark.sql.functions import cume_dist
from pyspark.sql.window import Window

sales_df1\
.withColumn("cume_dist", round(cume_dist().over(Window.partitionBy('CUSTOMERNAME').orderBy("SALES")), 5))\
.select("ORDERNUMBER", "CUSTOMERNAME", "SALES", "cume_dist")\
.show(50)

# INSIGHT: for the first row, cume_dist is 0.033 means that for the particular customer, 
# SALES value of that order is higher than 3.3% of all SALES values of the customer

+-----------+-------------------+-------+---------+
|ORDERNUMBER|       CUSTOMERNAME|  SALES|cume_dist|
+-----------+-------------------+-------+---------+
|      10141|Suominen Souveniers| 891.03|  0.03333|
|      10141|Suominen Souveniers| 1086.6|  0.06667|
|      10141|Suominen Souveniers|1103.76|      0.1|
|      10363|Suominen Souveniers|1629.04|  0.13333|
|      10247|Suominen Souveniers| 1988.4|  0.16667|
|      10141|Suominen Souveniers|2140.11|      0.2|
|      10363|Suominen Souveniers|2447.76|  0.23333|
|      10363|Suominen Souveniers|2632.89|  0.26667|
|      10363|Suominen Souveniers| 2773.8|      0.3|
|      10363|Suominen Souveniers|2775.08|  0.33333|
|      10363|Suominen Souveniers|2817.87|  0.36667|
|      10363|Suominen Souveniers|2851.84|      0.4|
|      10363|Suominen Souveniers|2931.98|  0.43333|
|      10247|Suominen Souveniers|3128.65|  0.46667|
|      10363|Suominen Souveniers|3288.82|      0.5|
|      10363|Suominen Souveniers|3595.62|  0.53333|
|      10363

### o) Write spark dataframe code to rank products by total revenue within each country (COUNTRY)?

In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank


sales_df1.withColumn(
    'REVENUESUM',col('QUANTITYORDERED')*col('PRICEEACH')
)\
.groupBy(['COUNTRY', 'PRODUCTLINE']).agg(round(sum('REVENUESUM'), 3).alias('TOTALREVENUE'))\
.withColumn('rank', dense_rank().over(Window.partitionBy('COUNTRY').orderBy(col('TOTALREVENUE').desc())))\
.show()

+-----------+----------------+------------+----+
|    COUNTRY|     PRODUCTLINE|TOTALREVENUE|rank|
+-----------+----------------+------------+----+
|     Sweden|    Classic Cars|    50377.62|   1|
|     Sweden|Trucks and Buses|    39562.44|   2|
|     Sweden|    Vintage Cars|    31784.94|   3|
|     Sweden|           Ships|    29514.62|   4|
|     Sweden|     Motorcycles|     12388.6|   5|
|     Sweden|          Planes|     7435.88|   6|
|     Sweden|          Trains|      3200.0|   7|
|Philippines|    Classic Cars|    43815.85|   1|
|Philippines|     Motorcycles|     17491.9|   2|
|Philippines|          Planes|    17048.33|   3|
|Philippines|    Vintage Cars|     1935.09|   4|
|  Singapore|    Classic Cars|    91791.76|   1|
|  Singapore|Trucks and Buses|    75797.19|   2|
|  Singapore|    Vintage Cars|     30221.0|   3|
|  Singapore|           Ships|    13065.74|   4|
|  Singapore|          Trains|    12934.21|   5|
|  Singapore|     Motorcycles|      4175.6|   6|
|    Germany|    Cla

### p) Calculate a running total of SALES for each customer and show the top 5 customers by this cumulative total?

In [25]:
from pyspark.sql.window import Window

sales_df1\
.groupBy(['CUSTOMERNAME'])\
.agg(round(sum('SALES'), 3).alias('TOTALSALES'))\
.withColumn(
        'cumulative_sum',
        sum('TOTALSALES')\
            .over(Window.orderBy('TOTALSALES')\
                .rowsBetween(Window.unboundedPreceding, Window.currentRow))
).orderBy('cumulative_sum', ascending=False).show(5)

+--------------------+----------+------------------+
|        CUSTOMERNAME|TOTALSALES|    cumulative_sum|
+--------------------+----------+------------------+
|Euro Shopping Cha...| 912294.11|     1.003262885E7|
|Mini Gifts Distri...| 654858.06|        9120334.74|
|Australian Collec...| 200995.41|        8465476.68|
|  Muscle Machine Inc| 197736.94|8264481.2700000005|
|   La Rochelle Gifts|  180124.9|        8066744.33|
+--------------------+----------+------------------+
only showing top 5 rows



### q) Find and handle Invalid and Outliers values in entire DataFrame. (Check for only continuous dataset).

In [26]:
# "ORDERLINENUMBER", "ORDERNUMBER", should be included? [coz not necessarily continuous]
col_list = ["QUANTITYORDERED", "PRICEEACH", "SALES", "QTR_ID", "MONTH_ID", "YEAR_ID", "MSRP"]


# creating a deepcopy for us to work on
new_dataframe = sales_df1.alias("new_dataframe")

# only from the columns we determined as continuous, drop NA values
new_dataframe = new_dataframe.dropna(subset=col_list)

# for each filed, filter out values outside threshold
for fieldName in col_list:
    q1 = new_dataframe.approxQuantile(fieldName, [0.25], relativeError=0.0001)[0]
    q3 = new_dataframe.approxQuantile(fieldName, [0.75], relativeError=0.0001)[0]

    minThreshold = q1 - 1.5*(q3-q1)
    maxThreshold = q3 + 1.5*(q3-q1)
    #print(f"{fieldName}, {minThreshold}, {maxThreshold}")

    new_dataframe = new_dataframe.filter((col(fieldName)>=minThreshold) & (col(fieldName)<=maxThreshold))

new_dataframe.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

In [27]:
#sales_df.printSchema()

### r) How would you cache a DataFrame containing sales data from the top 10 countries by sales to avoid recomputation in subsequent transformations? What persistence level (e.g. MEMORY_ONLY, MEMORY_AND_DISK) would you choose and why?

In [28]:
countries_top10 = sales_df1.groupBy(['COUNTRY'])\
.agg(round(sum('SALES'), 3).alias("TOTALSALES"))\
.orderBy(col('TOTALSALES').desc())\
.limit(10)

countries_top10.show()

+---------+----------+
|  COUNTRY|TOTALSALES|
+---------+----------+
|      USA|3627982.83|
|    Spain|1215686.92|
|   France|1110916.52|
|Australia|  630623.1|
|       UK| 478880.46|
|    Italy| 374674.31|
|  Finland| 329581.91|
|   Norway|  307463.7|
|Singapore| 288488.41|
|  Denmark| 245637.15|
+---------+----------+



In [29]:
# caching
countries_top10_cached = countries_top10.cache()

In [30]:
# (persistence)[https://sparkbyexamples.com/pyspark/pyspark-persist-in-detail/]
from pyspark import StorageLevel
countries_top10_persisted = countries_top10.persist(StorageLevel.MEMORY_ONLY)

# the size of data that we are smalling here is small [10 rows and 2 columns], which can sit in the memory
# MEMORY_ONLY_SER is not used as serialization and deserialization requires overhead in processing
# MEMORY_AND_DISK need to be used only when storage required is larger than available memory
# replication to more cluster notes is also not required

### s) How would you pivot the data to show PRODUCTLINE as columns and the total SALES for each ORDERDATE as the values? What are the implications of pivoting large datasets in Spark?

In [31]:
sales_df1.withColumn('ORDERDATE', to_timestamp(col('ORDERDATE'), 'MM/dd/yyyy H:mm'))\
.withColumn("ORDERDATE", col("ORDERDATE").cast('date'))\
.groupBy("ORDERDATE")\
.pivot("PRODUCTLINE")\
.agg(round(sum("SALES"), 3))\
.orderBy("ORDERDATE")\
.show()

# performing pivot operations on lagre datasets can cause increased memory consumption
# and higher processing time.
# number of new columns in the new dataframe will depend on unique values in the pivot column

+----------+------------+-----------+--------+--------+-------+----------------+------------+
| ORDERDATE|Classic Cars|Motorcycles|  Planes|   Ships| Trains|Trucks and Buses|Vintage Cars|
+----------+------------+-----------+--------+--------+-------+----------------+------------+
|2003-01-06|        null|       null|    null|    null|   null|            null|    12133.25|
|2003-01-09|        null|       null|    null|    null|   null|            null|    11432.34|
|2003-01-10|        null|       null|    null|    null|   null|            null|     6864.05|
|2003-01-29|     15263.7|       null|    null|    null|   null|         23041.1|     16397.2|
|2003-01-31|    25928.08|       null|    null|    null|4933.55|        13760.33|        null|
|2003-02-11|    20464.41|       null|    null|20452.04| 4330.1|            null|    13624.56|
|2003-02-17|        null|       null|39205.31| 6598.34|   null|            null|    10377.67|
|2003-02-24|        null|   25783.76|    null|    null|   nu

### t) How would you calculate the percentage growth of total sales month over month for each PRODUCTLINE using Spark DataFrame?

In [32]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

sales_df1\
.withColumn('ORDERDATE', to_timestamp(col('ORDERDATE'), 'MM/dd/yyyy H:mm'))\
.select(
    col('PRODUCTLINE'), year('ORDERDATE').alias('YEAR'), month('ORDERDATE').alias('MONTH'), col('SALES'))\
.groupBy(["PRODUCTLINE", "YEAR", "MONTH"]).agg(round(sum("SALES"), 3).alias("SALES"))\
.withColumn(
    'LAG', 
    lag('SALES', 1).over(Window.partitionBy("PRODUCTLINE").orderBy("PRODUCTLINE", "YEAR", "MONTH")))\
.withColumn(
    'PERCENTAGE_GROWTH', 
    round((col("SALES")-col("LAG"))*100/col("LAG"), 3).alias('PERCENTAGE_GROWTH'))\
.show(35)

+------------+----+-----+---------+---------+-----------------+
| PRODUCTLINE|YEAR|MONTH|    SALES|      LAG|PERCENTAGE_GROWTH|
+------------+----+-----+---------+---------+-----------------+
| Motorcycles|2003|    2| 25783.76|     null|             null|
| Motorcycles|2003|    3| 12639.15| 25783.76|           -50.98|
| Motorcycles|2003|    4| 23475.59| 12639.15|           85.737|
| Motorcycles|2003|    5| 22097.32| 23475.59|           -5.871|
| Motorcycles|2003|    6|  2642.01| 22097.32|          -88.044|
| Motorcycles|2003|    7| 37924.23|  2642.01|         1335.431|
| Motorcycles|2003|    8| 44164.91| 37924.23|           16.456|
| Motorcycles|2003|    9|  3155.58| 44164.91|          -92.855|
| Motorcycles|2003|   10| 64235.65|  3155.58|         1935.621|
| Motorcycles|2003|   11| 109345.5| 64235.65|           70.226|
| Motorcycles|2003|   12| 25431.88| 109345.5|          -76.742|
| Motorcycles|2004|    1| 41200.52| 25431.88|           62.003|
| Motorcycles|2004|    2|  49066.5| 4120

### u) How can you rebalance the data by partioning based on the COUNTRY column to ensure that large data partitions are avoided?

In [33]:
sales_df1_repartitioned = sales_df1.repartition(col("COUNTRY"))
sales_df1_repartitioned.rdd.getNumPartitions()

# repartition() helps distribute the dataframe across different partitions, thereby balancing
# the distribution of data. This helps improve parallel processing

200

### v) Suppose you have a smaller lookup table with customer details. How would you perform a broadcast join with the large sales_data_sample dataset to improve join performance? What are the key considerations when using broadcast joins?

In [34]:
from pyspark.sql.functions import broadcast


customer_lookup_table = spark.read.csv('file:///home/hadoop/Downloads/sales_data_customer_demographic.csv', header=True, inferSchema=True)

joined_df = sales_df1.join(broadcast(customer_lookup_table), sales_df1.CUSTOMERNAME==customer_lookup_table.CUSTOMERNAME, "inner")
joined_df.show()


# key considerations when using broadcast join
# - one of the dataframes must be small enough to fit in the memory of each worker node
# it makes the join operation much less resource intensive

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+--------------------+---------+-------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|        CUSTOMERNAME|ESTD_YEAR|NUM_EMPLOYEES|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------

### w) Create a UDF that categorizes the sales values (SALES) into custom buckets like “Low”, “Medium”, “High”. Apply this UDF to the DataFrame and calculate the count of orders in each category per COUNTRY.

In [35]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


q33 = sales_df1.approxQuantile("SALES", [0.33], relativeError=0.0001)[0]
q66 = sales_df1.approxQuantile("SALES", [0.66], relativeError=0.0001)[0]

def categorizeSales(sales):
    if sales <= q33:
        return 'Low'
    elif sales <= q66:
        return 'Medium'
    else:
        return 'High'

# register UDF
udf_categorizeSales = udf(categorizeSales, StringType())

sales_df1.withColumn("SALES_CATEGORY", udf_categorizeSales(col("SALES")))\
.groupBy("COUNTRY")\
.pivot("SALES_CATEGORY")\
.agg(count("SALES_CATEGORY"))\
.orderBy("COUNTRY")\
.show()

+-----------+----+---+------+
|    COUNTRY|High|Low|Medium|
+-----------+----+---+------+
|  Australia|  61| 67|    57|
|    Austria|  20| 16|    19|
|    Belgium|  11| 14|     8|
|     Canada|  18| 28|    24|
|    Denmark|  23| 18|    22|
|    Finland|  32| 25|    35|
|     France| 102|112|   100|
|    Germany|  21| 22|    19|
|    Ireland|   8|  7|     1|
|      Italy|  32| 38|    43|
|      Japan|  15| 16|    21|
|     Norway|  34| 30|    21|
|Philippines|   9|  7|    10|
|  Singapore|  30| 28|    21|
|      Spain| 119|112|   111|
|     Sweden|  20| 16|    21|
|Switzerland|  14|  5|    12|
|         UK|  36| 53|    55|
|        USA| 354|318|   332|
+-----------+----+---+------+



### x) Create a Python UDF to calculate discounts for specific product lines. For example, give a 10% discount for Classic Cars and 5% for Motorcycles. Apply this UDF to derive new discounted sales values.

In [47]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType


def discountSales(productLine):
    productLine = productLine.lower()
    if productLine == 'classic cars': return 0.10
    elif productLine == 'motorcycles': return 0.05
    elif productLine == 'planes': return 0.20
    elif productLine == 'ships': return 0.25
    elif productLine == 'trains': return 0.30
    elif productLine == 'trucks and buses': return 0.35
    elif productLine == 'vintage cars': return 0.40
    else: return 0.0

# register UDF
udf_discountSales = udf(discountSales, DoubleType())

sales_df1.withColumn("SALES_DISCOUNT", udf_discountSales(col("PRODUCTLINE")))\
.select("ORDERNUMBER", "CUSTOMERNAME", "PRODUCTLINE", "SALES", "SALES_DISCOUNT")\
.withColumn("DISCOUNTED_SALES", round(col("SALES")*(1-col("SALES_DISCOUNT")), 3))\
.show()

+-----------+--------------------+-----------+-------+--------------+----------------+
|ORDERNUMBER|        CUSTOMERNAME|PRODUCTLINE|  SALES|SALES_DISCOUNT|DISCOUNTED_SALES|
+-----------+--------------------+-----------+-------+--------------+----------------+
|      10107|   Land of Toys Inc.|Motorcycles| 2871.0|          0.05|         2727.45|
|      10121|  Reims Collectables|Motorcycles| 2765.9|          0.05|        2627.605|
|      10134|     Lyon Souveniers|Motorcycles|3884.34|          0.05|        3690.123|
|      10145|   Toys4GrownUps.com|Motorcycles| 3746.7|          0.05|        3559.365|
|      10159|Corporate Gift Id...|Motorcycles|5205.27|          0.05|        4945.007|
|      10168|Technics Stores Inc.|Motorcycles|3479.76|          0.05|        3305.772|
|      10180|Daedalus Designs ...|Motorcycles|2497.77|          0.05|        2372.882|
|      10188|        Herkku Gifts|Motorcycles|5512.32|          0.05|        5236.704|
|      10201|     Mini Wheels Co.|Motorcycl

### y) How would you set up an incremental loading mechanism for orders placed daily based on the ORDERDATE column? How can Spark checkpointing can be used with incremental load to ensure no data loss occurs during failures?

inorder to use incremental loading mechanism for processing orders placed daily (assuming data is sorted in order of ORDERDATE), we first need to keep track of the date till which order details have already been loaded. 
when there is new data with ORDERDATE greater than the tracked date, they will also need to be loaded and tracked date updated.
before an update is done, it is necessary to checkpoint the previous state. this is done inorder to ensure that a recovery operation can be performed incase of any issue with the newly updated data

In [ ]:
# directory to store checkpoints
spark.sparkContext.setCheckpointDir('/home/hadoop/Downloads/checkpoints')

# incrementally loading data
def loadData(old_df):
    new_df = spark.read.csv('file:///home/hadoop/Downloads/sales_data_sample.csv', header=True, inferSchema=True)
    updated_df = old_df.union(new_df)
    return updated_df

In [ ]:
try:
    new_sales_df = loadData(sales_df1)
    new_sales_df.checkpoint()
except Except

### z) How do you implement a cumulative distribution function (CDF) over the SALES value for each CUSTOMERNAME? What insights can you gather from analyzing the CDF distribution for each customer?

In [38]:
from pyspark.sql.functions import cume_dist
from pyspark.sql.window import Window

sales_df1\
.withColumn("cume_dist", round(cume_dist().over(Window.partitionBy('CUSTOMERNAME').orderBy("SALES")), 5))\
.select("ORDERNUMBER", "CUSTOMERNAME", "SALES", "cume_dist")\
.show(50)

# INSIGHT: for the first row, cume_dist is 0.033 means that for the particular customer, 
# SALES value of that order is higher than 3.3% of all SALES values of the customer

+-----------+-------------------+-------+---------+
|ORDERNUMBER|       CUSTOMERNAME|  SALES|cume_dist|
+-----------+-------------------+-------+---------+
|      10141|Suominen Souveniers| 891.03|  0.03333|
|      10141|Suominen Souveniers| 1086.6|  0.06667|
|      10141|Suominen Souveniers|1103.76|      0.1|
|      10363|Suominen Souveniers|1629.04|  0.13333|
|      10247|Suominen Souveniers| 1988.4|  0.16667|
|      10141|Suominen Souveniers|2140.11|      0.2|
|      10363|Suominen Souveniers|2447.76|  0.23333|
|      10363|Suominen Souveniers|2632.89|  0.26667|
|      10363|Suominen Souveniers| 2773.8|      0.3|
|      10363|Suominen Souveniers|2775.08|  0.33333|
|      10363|Suominen Souveniers|2817.87|  0.36667|
|      10363|Suominen Souveniers|2851.84|      0.4|
|      10363|Suominen Souveniers|2931.98|  0.43333|
|      10247|Suominen Souveniers|3128.65|  0.46667|
|      10363|Suominen Souveniers|3288.82|      0.5|
|      10363|Suominen Souveniers|3595.62|  0.53333|
|      10363